# AIFFEL 여덟번째 프로젝트 : Movielens 영화 추천 실습
- 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 **Movielens 데이터**

    - 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 존재
    - MovieLens 1M Dataset 사용을 권장
    - 별점 데이터는 대표적인 explicit 데이터지만 implicit 데이터로 간주하고 테스트 가능
    - 별점을 시청횟수로 해석
    - 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외

## 0. 데이터 다운
```
1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data
3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data
4) 압축 해제
$ unzip ml-1m.zip
```

## 1. 데이터 준비와 전처리
- Movielens 데이터는 `rating.dat` 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있다.

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()
ratings.tail()

,user_id,movie_id,rating,timestamp
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
# ratings['count']

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'ISO-8859-1')
movies.head()
# movies.tail()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2. 분석해 봅시다
- ratings에 있는 유니크한 영화 개수

In [6]:
ratings['movie_id'].nunique()

3628

- ratings에 있는 유니크한 사용자 수

In [7]:
ratings['user_id'].nunique()

6039

- 가장 인기 있는 영화 30개(인기순)
    - 가장 많이 본 영화로 인기를 판가름내기 위해, ratings, movies 데이터프레임을 병합한 movie 데이터프레임을 만들었다.

In [8]:
movie = pd.merge(ratings, movies, on='movie_id', how = 'left')
movie

movie_count = movie.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)
# movie_count

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.
### 모델 검증을 하기 위한 사용자 초기 정보 세팅
- 유투브, 왓챠 등 추천 시스템을 사용하는 프로그램에서는 추천 과정의 정확도를 높이기 위해, 사용자의 초기 정보를 세팅하는 과정을 거친다. 따라서 우리 프로젝트에서도 내가 좋아하는 영화의 정보를 다섯 가지 세팅해준다.
- ratings 데이터프레임과 movies 데이터프레임을 따로 사용하고 있기 때문에, movies 데이터프레임에서 정보를 가져와 ratings 데이터프레임에 저장하는 과정을 거쳤다.
- 내 아이디를 넣기 전 가장 마지막 아이디 넘버가 6040 이었기 때문에, 내 아이디를 6041로 설정하였다.

In [10]:
my_favorite = ['Pulp Fiction (1994)', 'Godfather, The (1972)', 'Matrix, The (1999)', 'Toy Story (1995)', 'E.T. the Extra-Terrestrial (1982)']
# my_favorite_id = movie.loc([x if  for title in movie.title]
my_rating = [4.0, 4.0, 4.0, 4.0, 4.0]

my_fav_id = []
for fav in my_favorite:
    movie_fav_id = movie.movie_id[movie.title==fav].drop_duplicates().values
    for movieid in movie_fav_id:
        my_fav_id.append(movieid)
my_fav_id

my_movielist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_fav_id, 'count': my_rating})
my_movielist

# ratings.drop(['rating'], axis = 1)
if not ratings.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,timestamp
1000203,6040,1090,3.0,956715518.0
1000205,6040,1094,5.0,956704887.0
1000206,6040,562,5.0,956704746.0
1000207,6040,1096,4.0,956715648.0
1000208,6040,1097,4.0,956715569.0
0,6041,296,4.0,NaN
1,6041,858,4.0,NaN
2,6041,2571,4.0,NaN
3,6041,1,4.0,NaN
4,6041,1097,4.0,NaN


## 4. CSR matrix를 직접 만들어 봅시다.
### 모델 활용 전 데이터 전처리
- 데이터에서 유니크한 데이터를 통해 데이터 별로 *인덱싱* 과정을 거치게 된다. 인덱싱 과정은 데이터 관리를 쉽게 해주기 위해, 데이터 별로 번호를 붙여 주는 과정으로 추천 시스템, 자연어 처리에서 자주 진행하는 작업이다.

In [18]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique() # 특정 컬럼에 포함된 유니크한 데이터만 모아줌
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [25]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('artist column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('artist column indexing Fail!!')

ratings

user_id column indexing OK!!
artist column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5.0,978300760.0
1,0,1,3.0,978302109.0
2,0,2,3.0,978301968.0
3,0,3,4.0,978300275.0
4,0,4,5.0,978824291.0
...,...,...,...,...
0,6039,222,4.0,NaN
1,6039,607,4.0,NaN
2,6039,124,4.0,NaN
3,6039,40,4.0,NaN


In [40]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [41]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [42]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [43]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [44]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [47]:
miri = user_to_idx[6041]
miri_vector = als_model.user_factors[miri]

In [60]:
pulp = movie_to_idx[2571]
pulp_vector = als_model.item_factors[pulp]

toy = movie_to_idx[1]
toy_vector = als_model.item_factors[toy]
np.dot(miri_vector, pulp_vector), np.dot(miri_vector, toy_vector)

(0.3822459, 0.4402602)

- 두 영화의 평점을 같게 뒀고, 4점인 높은 점수였지만 그닥 선호도가 크게 나오지 않는 것을 볼 수 있다. 모델을 더 다듬거나, 데이터프레임에 시청 횟수까지 포함한다면 더 좋은 선호도 결과가 나올 것이다.

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [66]:
# favorite_movie = 'Pulp Fiction (1994)'
favorite_movie = 296
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
similar_idx = [idx_to_movie[i[0]] for i in similar_movie]

similar_movie = []ㅠ
for idx in similar_idx:
    movie_title = movies.title[movies.movie_id==idx].values
    movie_genre = movies.genre[movies.movie_id==idx].values
    for movieti in zip(movie_title, movie_genre):
        similar_movie.append(movieti)
similar_movie

[('Pulp Fiction (1994)', 'Crime|Drama'),
 ('GoodFellas (1990)', 'Crime|Drama'),
 ('Fargo (1996)', 'Crime|Drama|Thriller'),
 ('Shawshank Redemption, The (1994)', 'Drama'),
 ('Usual Suspects, The (1995)', 'Crime|Thriller'),
 ('Silence of the Lambs, The (1991)', 'Drama|Thriller'),
 ('Reservoir Dogs (1992)', 'Crime|Thriller'),
 ('Good Will Hunting (1997)', 'Drama'),
 ('L.A. Confidential (1997)', 'Crime|Film-Noir|Mystery|Thriller'),
 ('Seven (Se7en) (1995)', 'Crime|Thriller'),
 ("Schindler's List (1993)", 'Drama|War'),
 ('Jackie Brown (1997)', 'Crime|Drama'),
 ('Trainspotting (1996)', 'Drama'),
 ('Condition Red (1995)', 'Action|Drama|Thriller'),
 ('Few Good Men, A (1992)', 'Crime|Drama')]

- 나는 쿠엔틴 타란티노의 영화를 좋아한다. 그의 영화의 대부분은 범죄 장르에 속하는데, 그 영화와 비슷한 장르 혹은 그 감독의 영화도 몇 가지 추천되는 것을 보아 합리적인 결과가 나왔음을 볼 수 있다. 다른 장르에 관한 결과를 파악하고자 애니메이션 영화와 유사한 영화를 추천 받아보았다.

In [68]:
# favorite_movie = 'Toy Story (1995)'
favorite_movie = 1
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
similar_idx = [idx_to_movie[i[0]] for i in similar_movie]

similar_movie = []
for idx in similar_idx:
    movie_title = movies.title[movies.movie_id==idx].values
    movie_genre = movies.genre[movies.movie_id==idx].values
    for movieti in zip(movie_title, movie_genre):
        similar_movie.append(movieti)
similar_movie

[('Toy Story (1995)', "Animation|Children's|Comedy"),
 ('Toy Story 2 (1999)', "Animation|Children's|Comedy"),
 ("Bug's Life, A (1998)", "Animation|Children's|Comedy"),
 ('Aladdin (1992)', "Animation|Children's|Comedy|Musical"),
 ('Babe (1995)', "Children's|Comedy|Drama"),
 ('Groundhog Day (1993)', 'Comedy|Romance'),
 ('Lion King, The (1994)', "Animation|Children's|Musical"),
 ('Pleasantville (1998)', 'Comedy'),
 ('Beauty and the Beast (1991)', "Animation|Children's|Musical"),
 ('Shakespeare in Love (1998)', 'Comedy|Romance'),
 ("There's Something About Mary (1998)", 'Comedy'),
 ('Mulan (1998)', "Animation|Children's"),
 ('Forrest Gump (1994)', 'Comedy|Romance|War'),
 ('Hercules (1997)', "Adventure|Animation|Children's|Comedy|Musical"),
 ('Full Monty, The (1997)', 'Comedy')]

- 픽사 제작 영화를 좋아하는데, 애니메이션 강자 디즈니 영화가 많이 나오는 것을 볼 수 있다. 애니메이션 장르 특성상 아이들 영화이거나 코미디 장르가 혼합되어 있기 때문에, 그와 비슷한 영화를 추천하는 것으로 봐서 이 결과도 합리적임을 알 수 있다.

## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [55]:
user = user_to_idx[6041]

# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(380, 0.50341654),
 (121, 0.37386847),
 (92, 0.3524468),
 (51, 0.3438506),
 (50, 0.3436462),
 (269, 0.34294826),
 (157, 0.31839952),
 (317, 0.29181096),
 (117, 0.2781814),
 (48, 0.2746763),
 (44, 0.27399457),
 (322, 0.25542983),
 (435, 0.25068626),
 (224, 0.24238707),
 (4, 0.23891158),
 (200, 0.23650926),
 (233, 0.23047736),
 (38, 0.22470817),
 (479, 0.21657082),
 (110, 0.21223934)]

In [65]:
recom_idx = [idx_to_movie[i[0]] for i in movie_recommended]

recom_movie = []
for idx in recom_idx:
    recom_title = movies.title[movies.movie_id==idx].values
    recom_genre = movies.genre[movies.movie_id==idx].values
    for title_g in zip(recom_title, recom_genre):
        recom_movie.append(title_g)
recom_movie

[('Godfather: Part II, The (1974)', 'Action|Crime|Drama'),
 ('Silence of the Lambs, The (1991)', 'Drama|Thriller'),
 ('Terminator 2: Judgment Day (1991)', 'Action|Sci-Fi|Thriller'),
 ('Fargo (1996)', 'Crime|Drama|Thriller'),
 ('Toy Story 2 (1999)', "Animation|Children's|Comedy"),
 ('GoodFellas (1990)', 'Crime|Drama'),
 ('Shawshank Redemption, The (1994)', 'Drama'),
 ('Twelve Monkeys (1995)', 'Drama|Sci-Fi'),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)',
  'Action|Adventure|Drama|Sci-Fi|War'),
 ('Saving Private Ryan (1998)', 'Action|Drama|War'),
 ('Star Wars: Episode IV - A New Hope (1977)',
  'Action|Adventure|Fantasy|Sci-Fi'),
 ('Babe (1995)', "Children's|Comedy|Drama"),
 ('Godfather: Part III, The (1990)', 'Action|Crime|Drama'),
 ('L.A. Confidential (1997)', 'Crime|Film-Noir|Mystery|Thriller'),
 ("Bug's Life, A (1998)", "Animation|Children's|Comedy"),
 ('Terminator, The (1984)', 'Action|Sci-Fi|Thriller'),
 ('Usual Suspects, The (1995)', 'Crime|Thriller'),
 ('Sixth Sense,

- 내가 좋아하는 장르를 함께 파악하려 리스트에 장르도 추가했다. 좋아하는 영화 장르가 범죄, 스릴러, SF 아니면 애니메이션이라 추천 결과가 두 세개 외에는 보고싶어하던 영화가 결과로 나왔다.

---
# 루브릭
## 1. CSR matrix가 정상적으로 만들어졌다.
- 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
- 각종 전처리 과정을 거쳐, 사용자 및 아이템 개수를 통해 CSR 행렬을 만들었다. 다른 노드도 그렇고 항상 느끼는 것이지만 **전처리 과정이 가장 오래 걸리고 가장 중요한 과정**임을 또 한 번 느꼈다.

## 2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
- 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
- 각 영화에 대한 선호도 값(벡터 내적값)은 낮았지만, 전체 선호도 자체가 크게 높은 편이 아니었기 때문에, 추천에 대한 결과는 지금 사용하고 있는 왓챠와 비슷하게 나온 편이다. 모델을 더 다듬거나 결과에 영향을 줄만한 데이터 요소들을 더 넣게 되면 더 좋은 결과가 나올 것이다.

## 3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.
- MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.
- 내가 좋아할만한 영화 추천과 나의 선호도 순서 등의 수치가 의미있게 측정되었다. 루브릭 2와 같은 결과로 장르 추천만 보아도 좋아하는 영화의 선호도가 높게 측정되며 좋은 결과가 도출되었다.

---
# 마무리
2021 02 16 TUE
- 연휴를 맞아 놀았더니 또 과제가 밀려 있다. 두 데이터프레임을 합해서 CSR matrix나 모델 진행을 했으면, 굳이 중간 코드 작성 없이 영화 타이틀을 바로 보여주게끔 할 수 있었다. CSR matrix에서 오류가 날 것 같아 추가적으로 점차 수정하려 한다.
- rating 컬럼을 count로 변경하는 과정은 굳이 필요하지 않은 것 같다. 메소드로 사용하고 있는 단어이기 때문에, 오류가 생길 확률이 많다. 변수 네이밍의 중요성!